#Best Buy SKU Classifier

The point of this python notebook is to see how accurate KNN and Naive Bayes are to choosing skus that that consumers will choose due to past behavior


# Import Library and CSV File

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
clean_data = pd.read_csv('/content/sample_data/bestbuysmalldataset2.csv')
#raw_data.head()

#raw_data.drop(raw_data.columns[len(raw_data.columns)-1],axis=1,inplace = True)
display(clean_data)

,user,sku,category,query,click_time,query_time
0,000000df17cd56a5df4a94074e133c9d4739fae3,2125233.0,abcat0101001,Televisiones Panasonic 50 pulgadas,44:52.5,43:59.8
1,000001928162247ffaf63185cd8b2a244c78e7c6,2009324.0,abcat0101001,Sharp,25:37.4,25:01.2
2,000017f79c2b5da56721f22f9fdd726b13daf8e8,1517163.0,pcmcat193100050014,nook,56:58.9,55:13.0
3,000017f79c2b5da56721f22f9fdd726b13daf8e8,2877125.0,abcat0101001,rca,18:14.7,16:51.8
4,000017f79c2b5da56721f22f9fdd726b13daf8e8,2877134.0,abcat0101005,rca,19:51.7,16:51.8
...,...,...,...,...,...,...
201050,1bc5408c8833aa72ed7fbc27a517af6f07ff32d5,7818631.0,cat02015,dvd player,21:26.6,20:36.2
201051,1bc544a20d4a874415c1f36cbb54b35e019ddd75,2899309.0,abcat0513004,sony laptops,30:29.1,28:53.8
201052,1bc54da313c7c0c368f86880c848509baed5bddc,417524.0,cat02673,Metallica,27:00.0,24:40.9
201053,1bc54da313c7c0c368f86880c848509baed5bddc,377149.0,cat02673,Metallica,28:08.7,24:40.9


# Data Visualization

In [ ]:
##clean_data['category'].value_counts().plot(kind='bar')
##plt.title('Number of Category', fontsize=20)
##plt.xlabel('Category', fontsize=13)
##plt.ylabel("Total Numbers", fontsize=13)
clean_data['category'].value_counts()
clean_data['sku'].value_counts()

2842056.0    2535
2842092.0    1405
1945531.0    1343
9755322.0    1261
9225377.0    1241
             ... 
3004308.0       1
7978101.0       1
3469549.0       1
6939076.0       1
1048591.0       1
Name: sku, Length: 28182, dtype: int64

# Adding Features for Training data

In [ ]:
clean_data['Categoryid'] = pd.factorize(clean_data.category)[0]

def clicktime(row):
  j = row.click_time
  if type(j) == float:
    display(row) 
  j =j.replace(':', '.')
  j = j[0:5]
  d = float(j)
  return j

def querytime(row):
  j = row.query_time
  if type(j) == float:
    display(row) 
  j =j.replace(':', '.')
  j = j[0:5]
  d = float(j)
  return j
clean_data['Click_time_int'] = clean_data.apply(clicktime,axis=1)
clean_data['Query_time_float'] = clean_data.apply(querytime,axis=1)
display(clean_data)

,user,sku,category,query,click_time,query_time,Categoryid,Click_time_int,Query_time_float
0,000000df17cd56a5df4a94074e133c9d4739fae3,2125233.0,abcat0101001,Televisiones Panasonic 50 pulgadas,44:52.5,43:59.8,0,44.52,43.59
1,000001928162247ffaf63185cd8b2a244c78e7c6,2009324.0,abcat0101001,Sharp,25:37.4,25:01.2,0,25.37,25.01
2,000017f79c2b5da56721f22f9fdd726b13daf8e8,1517163.0,pcmcat193100050014,nook,56:58.9,55:13.0,1,56.58,55.13
3,000017f79c2b5da56721f22f9fdd726b13daf8e8,2877125.0,abcat0101001,rca,18:14.7,16:51.8,0,18.14,16.51
4,000017f79c2b5da56721f22f9fdd726b13daf8e8,2877134.0,abcat0101005,rca,19:51.7,16:51.8,2,19.51,16.51
...,...,...,...,...,...,...,...,...,...
201050,1bc5408c8833aa72ed7fbc27a517af6f07ff32d5,7818631.0,cat02015,dvd player,21:26.6,20:36.2,20,21.26,20.36
201051,1bc544a20d4a874415c1f36cbb54b35e019ddd75,2899309.0,abcat0513004,sony laptops,30:29.1,28:53.8,40,30.29,28.53
201052,1bc54da313c7c0c368f86880c848509baed5bddc,417524.0,cat02673,Metallica,27:00.0,24:40.9,50,27.00,24.40
201053,1bc54da313c7c0c368f86880c848509baed5bddc,377149.0,cat02673,Metallica,28:08.7,24:40.9,50,28.08,24.40


In [ ]:
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

y = clean_data.sku
X = clean_data.drop(['sku'], axis = 1)
Z = X.drop(['query','user','category','click_time','query_time'],1)
X_train,X_test,y_train,y_test= train_test_split(Z,y,test_size=0.2, random_state=2)
display(X_train)
display(clean_data.category)

,Categoryid,Click_time_int,Query_time_float
77160,20,00.57,59.28
191970,74,07.33,07.04
114492,76,28.59,28.03
79875,6,07.02,06.50
60974,32,21.42,20.22
...,...,...,...
162091,311,32.31,31.05
175638,82,29.25,28.28
95816,210,23.15,22.33
100879,328,36.44,36.13


0               abcat0101001
1               abcat0101001
2         pcmcat193100050014
3               abcat0101001
4               abcat0101005
                 ...        
201050              cat02015
201051          abcat0513004
201052              cat02673
201053              cat02673
201054    pcmcat209000050008
Name: category, Length: 201055, dtype: object

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import time
from sklearn.preprocessing import LabelEncoder
from sklearn import utils
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
starttime1 = time.time()
clf_dt = DecisionTreeClassifier()
clf_dt.fit(X_train, y_train)
endtime1 = time.time()
decisionpre = clf_dt.predict(X_test)
#target3 = clf_dt.predict(X_train)
#finalscore=f1_score(y_test, target2, average='macro'), sum(y_test == target2) / float(len(target2))
#f1_score(y_train, target3, average='macro')#, sum(y_train == target3) / float(len(target3))
accuracy2 = clf_dt.score(X_test,y_test)
print("Accuracy: ", accuracy2)
print(f"Training time: {endtime1 - starttime1}s")
print(decisionpre)
#probabilitydt = clf_dt.predict_proba(X_test)
#probabilitydt = pd.DataFrame(probabilitydt, columns=['Strength 0', 'Strength 1','Strength 2'])
#display((probabilitydt).head(10))

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
import time
from sklearn.tree import DecisionTreeClassifier
import time
from sklearn.preprocessing import LabelEncoder
from sklearn import utils
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
clf_nb = GaussianNB()
starttime2 = time.time()
clf_nb.fit(X_train,y_train)
endtime2 = time.time()

accuracy3 = clf_nb.score(X_test,y_test)
print("Accuracy: " , accuracy3)
print(f"Training time: {endtime2 - starttime2}s")
print(clf_nb.predict(X_test))
#probabilitynb = clf_nb.predict_proba(X_test)
#probabilitynb = pd.DataFrame(probabilitynb, columns=['Strength 0', 'Strength 1','Strength 2'])
#display((probabilitynb).head(10))

Accuracy:  0.15488299221606028
Training time: 16.786591053009033s


KeyboardInterrupt: ignored